In [4]:
from src.pyfunc_onnx import ONNXrinnaGPT2

In [2]:
import os
cache_dir = os.path.join(".", "cache_models")
model_path = os.path.join(cache_dir, "rinna_gpt2_beam_step_search_optimized_gpt2_int8.onnx")

onnx_gpt2 = ONNXrinnaGPT2(model_path)

In [3]:
onnx_gpt2.predict('私はりんな')

Text generation using OnnxRuntime ...


['私はりんな @audio_snowfox @audio_snowfox @audio_snowfox @au',
 '私はりんな @audio_snowfox ありがとうございます。',
 '私はりんな @audio_snowfox @audio_snowfox @audio_snowfox rt',
 '私はりんな @audio_snowfox ありがとうございます! @audio_snowfox ありがとうございます!']

In [5]:
import os
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
import mlflow

interactive_auth = InteractiveLoginAuthentication(tenant_id="72f988bf-86f1-41af-91ab-2d7cd011db47")
ws = Workspace.from_config(path='config.json',auth=interactive_auth)

mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

experiment_name = "rinna-gpt2-exp"
mlflow.set_experiment(experiment_name)

In [6]:
conda_env = {
    'name': 'mlflow-env',
    'channels': ['conda-forge'],
    'dependencies': [
        'python=3.8.10',
        'pip',
        {'pip': [
            'mlflow',
            'cloudpickle==1.6.0',
            'numpy',
            'torch==1.9.0+cpu',
            'torchvision==0.10.0+cpu',
            'onnxruntime==1.8.1',
            'sentencepiece',
            'transformers==4.8.2',
            'onnx',
            'onnxconverter_common',
            'psutil',
            'pytz',
            'pandas',
            'py-cpuinfo',
            'py3nvml',
            'sympy', 
            'coloredlogs',
            'azureml-core',
            'azureml-mlflow'
            ]
        }
    ]
}

In [7]:
import mlflow
import mlflow.pytorch

client = mlflow.tracking.MlflowClient()

registered_model = client.get_model_version(name='test-model',version=37)

In [ ]:
remote_model_path = os.path.join('outputs','onnx')
mlflow.log_artifact(model_path, remote_model_path)
model_uri = "runs:/{}/".format(registered_model.run_id) + remote_model_path # run_id を持っておけば with mlflow.start_run() as run は不要
#mlflow.register_model(model_uri, 'rinna-GPT2-quantized_int8-model')
mlflow.pyfunc.log_model(artifact_path='onnx/', python_model=onnx_gpt2, conda_env=conda_env, registered_model_name='onnx_beam')